# Area Source Modeling with PyAERMOD

This notebook demonstrates area source modeling using the newly implemented AREA, AREACIRC, and AREAPOLY source types.

## Topics Covered

1. Rectangular area sources (AREA) - Storage piles, parking lots
2. Circular area sources (AREACIRC) - Tank farms, circular facilities
3. Polygonal area sources (AREAPOLY) - Irregular facility boundaries
4. Mixed source types (point + area)
5. Emission rate calculations (g/s/m²)

**Prerequisites:**
- PyAERMOD installed with area source support
- Understanding of area source emission rates (mass/time/area)
- Meteorological data files

## Setup

In [ ]:
# Import PyAERMOD with area source support
from pyaermod.input_generator import (
    AERMODProject,
    ControlPathway,
    SourcePathway,
    PointSource,
    AreaSource,
    AreaCircSource,
    AreaPolySource,
    ReceptorPathway,
    CartesianGrid,
    MeteorologyPathway,
    OutputPathway,
    PollutantType,
    TerrainType
)

import numpy as np
import math

print("✓ PyAERMOD modules imported successfully")
print("✓ Area source support enabled")

## Understanding Area Source Emission Rates

Unlike point sources (g/s), area sources use **emission rate per unit area** (g/s/m²).

### Example Calculation

Storage pile: 100m × 50m = 5,000 m²  
Total emissions: 0.25 g/s  
Emission rate = 0.25 g/s ÷ 5,000 m² = **0.00005 g/s/m²**

Very small numbers are typical for area sources!

In [ ]:
def calculate_area_emission_rate(total_emission_gs, area_m2):
    """Convert total emission rate to area-specific rate"""
    return total_emission_gs / area_m2

# Example
pile_length = 100  # m
pile_width = 50    # m
pile_area = pile_length * pile_width
total_emissions = 0.25  # g/s

area_rate = calculate_area_emission_rate(total_emissions, pile_area)
print(f"Storage pile: {pile_length}m × {pile_width}m = {pile_area:,} m²")
print(f"Total emissions: {total_emissions} g/s")
print(f"Area emission rate: {area_rate:.8f} g/s/m²")
print(f"Scientific notation: {area_rate:.2e} g/s/m²")

## Example 1: Rectangular Area Source (Storage Pile)

Model fugitive dust from a coal storage pile.

In [ ]:
control = ControlPathway(
    title_one="Rectangular Area Source",
    title_two="Coal storage pile fugitive emissions",
    pollutant_id=PollutantType.PM10,
    averaging_periods=["24", "ANNUAL"],
    terrain_type=TerrainType.FLAT
)

sources = SourcePathway()

# Storage pile dimensions
pile_half_width_x = 25.0   # Half-width in X direction (50m total width)
pile_half_width_y = 50.0   # Half-width in Y direction (100m total length)
pile_area = (2 * pile_half_width_x) * (2 * pile_half_width_y)
total_emissions = 0.5  # g/s total
emission_rate = total_emissions / pile_area

print(f"Pile dimensions: {pile_half_width_x*2}m × {pile_half_width_y*2}m")
print(f"Total area: {pile_area} m²")
print(f"Emission rate: {emission_rate:.8f} g/s/m²\n")

sources.add_source(AreaSource(
    source_id="COALPILE",
    x_coord=0.0,
    y_coord=0.0,
    base_elevation=10.0,
    release_height=2.0,  # 2m above ground (pile height effect)
    initial_lateral_dimension=pile_half_width_x,  # Half-width X
    initial_vertical_dimension=pile_half_width_y,  # Half-width Y
    emission_rate=emission_rate,
    angle=0.0,  # No rotation
    source_groups=["ALL", "FUGITIVE"]
))

# Receptor grid
receptors = ReceptorPathway()
receptors.add_cartesian_grid(
    CartesianGrid.from_bounds(
        x_min=-500,
        x_max=500,
        y_min=-500,
        y_max=500,
        spacing=25
    )
)

meteorology = MeteorologyPathway(
    surface_file="met.sfc",
    profile_file="met.pfl"
)

output = OutputPathway(
    receptor_table=True,
    max_table=True
)

project = AERMODProject(
    control=control,
    sources=sources,
    receptors=receptors,
    meteorology=meteorology,
    output=output
)

project.write("area_rectangular_pile.inp")
print("✓ Created: area_rectangular_pile.inp")

## Example 2: Rotated Rectangular Area

The `angle` parameter rotates the area source (useful for non-N/S aligned features).

In [ ]:
control = ControlPathway(
    title_one="Rotated Area Source",
    title_two="Parking lot aligned to roadway",
    pollutant_id=PollutantType.PM25,
    averaging_periods=["ANNUAL"],
    terrain_type=TerrainType.FLAT
)

sources = SourcePathway()

# Parking lot: 150m × 75m, rotated 30° to match roadway
sources.add_source(AreaSource(
    source_id="PARKING",
    x_coord=100.0,
    y_coord=50.0,
    base_elevation=5.0,
    release_height=0.5,  # Very low - vehicle exhaust at ground level
    initial_lateral_dimension=37.5,  # 75m / 2
    initial_vertical_dimension=75.0,  # 150m / 2
    emission_rate=0.00001,  # Low rate - idling vehicles
    angle=30.0,  # Rotated 30 degrees
    source_groups=["ALL", "MOBILE"]
))

receptors = ReceptorPathway()
receptors.add_cartesian_grid(
    CartesianGrid.from_bounds(
        x_min=-200,
        x_max=400,
        y_min=-200,
        y_max=300,
        spacing=25
    )
)

meteorology = MeteorologyPathway(
    surface_file="met.sfc",
    profile_file="met.pfl"
)

output = OutputPathway(
    receptor_table=True,
    max_table=True
)

project = AERMODProject(
    control=control,
    sources=sources,
    receptors=receptors,
    meteorology=meteorology,
    output=output
)

project.write("area_rotated_parking.inp")
print("✓ Created: area_rotated_parking.inp")
print("\nNote: Rotation is clockwise from north (0° = north, 90° = east)")

## Example 3: Circular Area Source (Tank Farm)

Use `AreaCircSource` for circular facilities.

In [ ]:
control = ControlPathway(
    title_one="Circular Area Source",
    title_two="Chemical storage tank farm",
    pollutant_id="VOC",
    averaging_periods=["1", "ANNUAL"],
    terrain_type=TerrainType.FLAT
)

sources = SourcePathway()

# Tank farm: 100m radius circle
radius = 100.0  # meters
circle_area = math.pi * radius**2
total_voc_emissions = 2.0  # g/s from all tanks
emission_rate = total_voc_emissions / circle_area

print(f"Tank farm radius: {radius} m")
print(f"Total area: {circle_area:.1f} m²")
print(f"Total VOC emissions: {total_voc_emissions} g/s")
print(f"Emission rate: {emission_rate:.8f} g/s/m²\n")

sources.add_source(AreaCircSource(
    source_id="TANKFARM",
    x_coord=0.0,
    y_coord=0.0,
    base_elevation=5.0,
    release_height=3.0,  # Average tank vent height
    radius=radius,
    emission_rate=emission_rate,
    num_vertices=36,  # More vertices = smoother circle (computational cost)
    source_groups=["ALL", "TANKS", "VOC"]
))

receptors = ReceptorPathway()
receptors.add_cartesian_grid(
    CartesianGrid.from_bounds(
        x_min=-500,
        x_max=500,
        y_min=-500,
        y_max=500,
        spacing=25
    )
)

meteorology = MeteorologyPathway(
    surface_file="met.sfc",
    profile_file="met.pfl"
)

output = OutputPathway(
    receptor_table=True,
    max_table=True
)

project = AERMODProject(
    control=control,
    sources=sources,
    receptors=receptors,
    meteorology=meteorology,
    output=output
)

project.write("area_circular_tankfarm.inp")
print("✓ Created: area_circular_tankfarm.inp")
print("\nNote: num_vertices determines circle smoothness (20-40 typical)")

## Example 4: Polygonal Area Source (Irregular Facility)

Use `AreaPolySource` for irregular shapes defined by vertices.

In [ ]:
control = ControlPathway(
    title_one="Polygonal Area Source",
    title_two="Irregular facility boundary",
    pollutant_id=PollutantType.PM25,
    averaging_periods=["24", "ANNUAL"],
    terrain_type=TerrainType.FLAT
)

sources = SourcePathway()

# Define facility boundary as list of (x, y) vertices
# Vertices should form a closed polygon (AERMOD closes automatically)
facility_vertices = [
    (0.0, 0.0),      # Southwest corner
    (150.0, 0.0),    # Southeast corner
    (200.0, 80.0),   # Northeast extension
    (180.0, 150.0),  # North point
    (80.0, 120.0),   # Northwest
    (-30.0, 70.0),   # West indent
]

print("Facility polygon vertices:")
for i, (x, y) in enumerate(facility_vertices, 1):
    print(f"  {i}. ({x:7.1f}, {y:7.1f})")

# Calculate approximate area using shoelace formula
def polygon_area(vertices):
    n = len(vertices)
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += vertices[i][0] * vertices[j][1]
        area -= vertices[j][0] * vertices[i][1]
    return abs(area) / 2.0

poly_area = polygon_area(facility_vertices)
total_emissions = 1.5  # g/s
emission_rate = total_emissions / poly_area

print(f"\nPolygon area: {poly_area:.1f} m²")
print(f"Total emissions: {total_emissions} g/s")
print(f"Emission rate: {emission_rate:.8f} g/s/m²\n")

sources.add_source(AreaPolySource(
    source_id="FACILITY",
    vertices=facility_vertices,
    base_elevation=10.0,
    release_height=1.5,  # Low-level fugitive emissions
    emission_rate=emission_rate,
    source_groups=["ALL", "FACILITY"]
))

# Receptor grid covering facility and beyond
receptors = ReceptorPathway()
receptors.add_cartesian_grid(
    CartesianGrid.from_bounds(
        x_min=-200,
        x_max=400,
        y_min=-100,
        y_max=300,
        spacing=25
    )
)

meteorology = MeteorologyPathway(
    surface_file="met.sfc",
    profile_file="met.pfl"
)

output = OutputPathway(
    receptor_table=True,
    max_table=True
)

project = AERMODProject(
    control=control,
    sources=sources,
    receptors=receptors,
    meteorology=meteorology,
    output=output
)

project.write("area_polygon_facility.inp")
print("✓ Created: area_polygon_facility.inp")
print("\nNote: Vertices can define any irregular shape!")

## Example 5: Mixed Source Types (Point + Area)

Real facilities often have both point sources (stacks) and area sources (fugitives).

In [ ]:
control = ControlPathway(
    title_one="Mixed Source Facility",
    title_two="Point and area sources combined",
    pollutant_id=PollutantType.PM25,
    averaging_periods=["ANNUAL", "24"],
    terrain_type=TerrainType.FLAT
)

sources = SourcePathway()

# 1. Main stack (point source)
sources.add_source(PointSource(
    source_id="BOILER",
    x_coord=50.0,
    y_coord=50.0,
    base_elevation=10.0,
    stack_height=45.0,
    stack_temp=400.0,
    exit_velocity=15.0,
    stack_diameter=2.0,
    emission_rate=2.5,  # g/s
    source_groups=["ALL", "STACKS"]
))

# 2. Material handling area (rectangular area source)
sources.add_source(AreaSource(
    source_id="HANDLING",
    x_coord=-50.0,
    y_coord=-30.0,
    base_elevation=10.0,
    release_height=5.0,
    initial_lateral_dimension=20.0,  # 40m wide
    initial_vertical_dimension=25.0,  # 50m long
    emission_rate=0.0001,  # g/s/m²
    source_groups=["ALL", "FUGITIVE"]
))

# 3. Storage area (circular area source)
sources.add_source(AreaCircSource(
    source_id="STORAGE",
    x_coord=100.0,
    y_coord=-80.0,
    base_elevation=10.0,
    release_height=2.0,
    radius=40.0,
    emission_rate=0.00005,
    num_vertices=24,
    source_groups=["ALL", "FUGITIVE"]
))

# 4. Parking lot (rotated rectangular area)
sources.add_source(AreaSource(
    source_id="PARKING",
    x_coord=-100.0,
    y_coord=80.0,
    base_elevation=10.0,
    release_height=0.5,
    initial_lateral_dimension=30.0,
    initial_vertical_dimension=50.0,
    emission_rate=0.00002,
    angle=15.0,
    source_groups=["ALL", "MOBILE"]
))

print(f"✓ Defined {len(sources.sources)} sources:")
for src in sources.sources:
    src_type = src.__class__.__name__
    print(f"  - {src.source_id:12s} ({src_type})")

# Comprehensive receptor grid
receptors = ReceptorPathway()
receptors.add_cartesian_grid(
    CartesianGrid.from_bounds(
        x_min=-500,
        x_max=500,
        y_min=-500,
        y_max=500,
        spacing=25
    )
)

meteorology = MeteorologyPathway(
    surface_file="met.sfc",
    profile_file="met.pfl"
)

output = OutputPathway(
    receptor_table=True,
    max_table=True,
    summary_file="mixed_sources.sum"
)

project = AERMODProject(
    control=control,
    sources=sources,
    receptors=receptors,
    meteorology=meteorology,
    output=output
)

project.write("mixed_point_area_sources.inp")
print("\n✓ Created: mixed_point_area_sources.inp")
print("\nThis example shows realistic facility with:")
print("  • Elevated stack emission (point)")
print("  • Ground-level fugitive emissions (areas)")
print("  • Mobile source emissions (parking)")

## Example 6: Source Group Analysis

Use source groups to analyze contributions from different emission categories.

In [ ]:
# This example shows how to set up source groups for analysis
# AERMOD can report concentrations by group (ALL, STACKS, FUGITIVE, etc.)

control = ControlPathway(
    title_one="Source Group Analysis",
    title_two="Separate stack vs. fugitive contributions",
    pollutant_id=PollutantType.PM10,
    averaging_periods=["ANNUAL"],
    terrain_type=TerrainType.FLAT
)

sources = SourcePathway()

# Stack emissions - group "STACKS"
sources.add_source(PointSource(
    source_id="STACK1",
    x_coord=0.0,
    y_coord=0.0,
    base_elevation=10.0,
    stack_height=40.0,
    stack_temp=400.0,
    exit_velocity=12.0,
    stack_diameter=1.8,
    emission_rate=3.0,
    source_groups=["ALL", "STACKS"]  # Belongs to ALL and STACKS groups
))

# Fugitive emissions - group "FUGITIVE"
sources.add_source(AreaSource(
    source_id="PILE1",
    x_coord=80.0,
    y_coord=50.0,
    base_elevation=10.0,
    release_height=2.0,
    initial_lateral_dimension=20.0,
    initial_vertical_dimension=30.0,
    emission_rate=0.0002,
    source_groups=["ALL", "FUGITIVE"]  # Belongs to ALL and FUGITIVE groups
))

sources.add_source(AreaSource(
    source_id="ROAD",
    x_coord=-60.0,
    y_coord=-40.0,
    base_elevation=10.0,
    release_height=0.5,
    initial_lateral_dimension=5.0,
    initial_vertical_dimension=100.0,
    emission_rate=0.00015,
    source_groups=["ALL", "FUGITIVE"]  # Also FUGITIVE group
))

print("Source groups defined:")
print("  - ALL: Total facility emissions (all sources)")
print("  - STACKS: Point source contributions only")
print("  - FUGITIVE: Area source contributions only")
print("\nAERMOD will report concentrations for each group separately.")

receptors = ReceptorPathway()
receptors.add_cartesian_grid(
    CartesianGrid.from_bounds(
        x_min=-300,
        x_max=300,
        y_min=-300,
        y_max=300,
        spacing=25
    )
)

meteorology = MeteorologyPathway(
    surface_file="met.sfc",
    profile_file="met.pfl"
)

output = OutputPathway(
    receptor_table=True,
    max_table=True
)

project = AERMODProject(
    control=control,
    sources=sources,
    receptors=receptors,
    meteorology=meteorology,
    output=output
)

project.write("source_group_analysis.inp")
print("\n✓ Created: source_group_analysis.inp")
print("\nAnalyze output by source group to determine:")
print("  • Which emission category dominates?")
print("  • Where should control efforts focus?")
print("  • What's the benefit of stack emissions vs. fugitives?")

## Summary

This notebook demonstrated:

1. **Rectangular areas (AREA)** - Storage piles, parking lots, roads
2. **Circular areas (AREACIRC)** - Tank farms, circular facilities
3. **Polygonal areas (AREAPOLY)** - Irregular facility boundaries
4. **Mixed source types** - Combining point and area sources
5. **Source groups** - Analyzing contributions by category

### Key Concepts

- **Emission rates**: Area sources use g/s/m² (mass per time per area)
- **Dimensions**: Rectangular areas use "half-widths" in X and Y directions
- **Release height**: Affects initial vertical distribution
- **Rotation**: Angle parameter rotates rectangular areas
- **Vertices**: Polygons can have any shape (3+ vertices)
- **Source groups**: Enable contribution analysis

### Common Applications

| Source Type | Typical Applications |
|-------------|---------------------|
| **AREA** | Storage piles, roads, parking lots, yards |
| **AREACIRC** | Tank farms, lagoons, circular facilities |
| **AREAPOLY** | Irregular facility boundaries, treatment ponds |

### Next Steps

- **Notebook 04**: Parameter Sweeps and Batch Processing
- **Notebook 05**: Visualization Techniques
- **Notebook 06**: Volume Source Modeling (coming soon)